In [ ]:
import os

import matplotlib.pyplot as plt

from astropy.table import Table

%matplotlib notebook

In [ ]:
import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
from lsst.daf.persistence import Butler

In [ ]:
from deepcoadd_afw_to_astropy import read_cats

from wiyn_forcedPhotExternalCatalog import assemble_catalogs_into_lightcurve

In [ ]:
repo = os.path.join(os.getenv('DR1BASE'), 'repo', 'test_dr1')
rerun = os.path.join(repo, 'rerun', 'forcedPhot')

field, tract = 'LSQ13cwp', 0
# field = 'PTF11mty'

In [ ]:
J_cat, H_cat, ref_table = read_cats(field, tract=0, repo=rerun)

snr_threshold = 5
good_color = (J_cat['J_SNR'] > snr_threshold) & (H_cat['H_SNR'] > snr_threshold)

J_cat = J_cat[good_color]
H_cat = H_cat[good_color]
ref_table = ref_table[good_color]

In [ ]:
butler = Butler(rerun)

dId = {'field': field, 'filter': 'H', 'tract': 0, 'patch': '0,0'}
calexp = butler.get('deepCoadd', dataId=dId)

Find LSQ13cwp based on RA, Dec: 04:03:50.662  -02:39:18.57 (J2000)

In [ ]:
LSQ13cwp_RA_str, LSQ13cwp_Dec_str = '04:03:50.662', '-02:39:18.57'
LSQ13cwp_RA, LSQ13cwp_Dec = 60.961092, -2.655158

sn_coord = afwGeom.SpherePoint(LSQ13cwp_RA, LSQ13cwp_Dec, afwGeom.degrees)

distList = []
for s in ref_table:
    this_coord = afwGeom.SpherePoint(s['coord_ra'], s['coord_dec'], afwGeom.radians)
    angSep = sn_coord.separation(this_coord)
    distList.append(angSep)
    
distance = np.array(distList)
sn_idx = np.argmin(distance)

print("Found match: Object %d at %f arcsecs" % (sn_idx, afwGeom.radToArcsec(distance[sn_idx])))

Read in the forced-src photometry files that were built off of this same reference table to extract a lightcurve for LSQ13cwp

In [ ]:
# 2018-04-10:  MWV: This presently doesn't pull the correct expnums for each.
cat_dataRefs = butler.subset(datasetType='forced_src', dataId=dId)

In [ ]:
import copy
baseDataId = {'field': 'LSQ13cwp', 'seq': 'A'}
dataIds = [{'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'H', 'night': 20131111, 'expnum': 521},
           {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'H', 'night': 20131213, 'expnum': 274},
           {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'J', 'night': 20131111, 'expnum': 562},
           {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'J', 'night': 20131120, 'expnum': 425},
           {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'J', 'night': 20131209, 'expnum': 335},
           {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'J', 'night': 20131213, 'expnum': 299},
          ]

tract = 0
for dId in dataIds:
    dId['tract'] = tract
           

In [ ]:
dataIds_by_filter = {}
dataIds_by_filter['H'] = [
    {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'H', 'night': 20131111, 'expnum': 521},
    {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'H', 'night': 20131213, 'expnum': 274}
    ]
dataIds_by_filter['J'] = [
    {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'J', 'night': 20131111, 'expnum': 562},
    {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'J', 'night': 20131120, 'expnum': 425},
    {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'J', 'night': 20131209, 'expnum': 335},
    {'field': 'LSQ13cwp', 'seq': 'A', 'filter': 'J', 'night': 20131213, 'expnum': 299},
    ]

lc = assemble_catalogs_into_lightcurve(dataIds_by_filter, rerun, dataset='forced_src')
lc_file = '{:s}.ecsv'.format(field)
lc.write(lc_file, overwrite=True)

In [ ]:
lc.pprint(max_width=-1)

In [ ]:
filters = ['J', 'H']
colors = {'J': 'blue', 'H': 'green', 'KS': 'red'}
for filt in filters:
    wf, = np.where(lc['filter'] == filt)
    lc_filt = lc[wf]
    plt.errorbar(lc_filt['mjd'], lc_filt['base_PsfFlux_mag'], lc_filt['base_PsfFlux_magSigma'], label=filt,
                marker='o', color=colors[filt], linestyle='none')
    
    
ylim = plt.ylim()
plt.ylim(ylim[::-1])

plt.xlabel('MJD')
plt.ylabel('mag')
plt.legend()
plt.show()

In [ ]:
display = afwDisplay.getDisplay(backend='ds9')

In [ ]:
display.mtv(calexp)

In [ ]:
display.setMaskTransparency(80)
display.scale("asinh", -2, 25)

In [ ]:
X = 'slot_Centroid_x'
Y = 'slot_Centroid_y'

In [ ]:
display.erase()

In [ ]:
with display.Buffering():
    for s in ref_table:
        display.dot("o", s[X], s[Y], size=10, ctype='orange')

In [ ]:
sn_ref = ref_table[sn_idx]
display.dot("o", sn_ref[X], sn_ref[Y], size=20, ctype='green')